<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64


    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-01 09:26:00
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.37 C
-------------------
Today PnL: 41.54 K (0.3%)
Current PnL: -13.13 L (-9.15%)
CY Booked + Current PnL: -6.39 L (-4.45%)
-------------------
Total profit:  4.47 L
Total loss:  -17.61 L
-------------------
Total Booked + Current PnL: 20.59 L (17.66%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.93%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 71.56 L (52.32%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.93%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2980.00,NaN,NaN,78.12,74500.0,40051.0,0.0,0.14,116.26,0.00,116.26,NaN,inf,0.54,13.92,AR,ATH,ELECTRICAL
16,BLUSPRING,226.45,84.55,NaN,NaN,5.21,16631.0,-32962.0,1886.0,0.00,-66.46,11.34,-62.66,NaN,-17.48,0.12,11.53,XY24,ATH,MISC
28,DIGITIDE,188.38,237.72,NaN,NaN,1.04,43274.0,2019.0,8785.0,0.75,4.89,20.30,26.19,1.0,0.23,0.32,0.75,XY24,ATH,IT
3,ADANIPORTS,1103.69,1583.00,2.76,M-LC,26.04,103078.0,24716.0,9318.0,0.11,31.54,9.04,43.43,77.0,2.65,0.75,38.54,XY24,NTT,MISC
5,APOLLOHOSP,6904.43,8285.00,-16.85,H-LC,84.38,113332.0,9766.0,10948.0,4.33,9.43,9.66,20.00,30.0,0.89,0.83,24.83,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,86.99,M-MC,32.29,224859.0,-103.0,167767.0,-1.07,-0.05,74.61,74.53,185.0,-0.00,1.64,30.55,XY24,BTT,STEEL
55,LALPATHLAB,2804.19,3545.00,-33.15,H-SC,41.67,153967.0,-263.0,41001.0,0.11,-0.17,26.63,26.42,125.0,-0.01,1.13,21.42,X40N,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2337.55,-12.17,H-SC,71.88,164680.0,10.0,69067.0,0.68,0.01,41.94,41.95,121.0,0.00,1.20,7.47,X40N,ATH,AC
60,NATIONALUM,189.63,247.44,-50.15,H-MC,65.62,101002.0,498.0,30139.0,-1.11,0.50,29.84,30.49,97.0,0.02,0.74,33.06,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-49.70,L-SC,100.00,75744.0,-37805.0,77857.0,5.34,-33.29,102.79,35.27,261.0,-0.49,0.55,85.39,XR,NTT,HOTELS
5,APOLLOHOSP,6904.43,8285.00,-16.85,H-LC,84.38,113332.0,9766.0,10948.0,4.33,9.43,9.66,20.00,30.0,0.89,0.83,24.83,X40N,BTT,HEALTHCARE
54,KPIGREEN,497.21,732.16,29.90,H-SC,73.96,132917.0,7620.0,51585.0,2.49,6.08,38.81,47.25,139.0,0.15,0.97,66.29,MH,ATH,POWER
90,VALIANTORG,512.64,838.00,-356.39,H-SC,6.25,108875.0,-24411.0,109006.0,2.20,-18.31,100.12,63.47,148.0,-0.22,0.80,82.50,XR,NTT,CHEMICALS
7,ASIANTILES,75.41,137.00,7266.67,M-SC,89.58,80848.0,-12962.0,89580.0,1.87,-13.82,110.80,81.67,242.0,-0.14,0.59,56.56,XR,NTT,CERAMICS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,DIXON,14367.00,18940.15,-36.77,H-MC,29.17,116376.0,1440.0,35146.0,-2.91,1.25,30.20,31.83,99.0,0.04,0.85,33.38,X40N,ATH,IT
60,NATIONALUM,189.63,247.44,-50.15,H-MC,65.62,101002.0,498.0,30139.0,-1.11,0.50,29.84,30.49,97.0,0.02,0.74,33.06,MH,ATH,METALS
68,SAIL,130.64,228.00,86.99,M-MC,32.29,224859.0,-103.0,167767.0,-1.07,-0.05,74.61,74.53,185.0,-0.00,1.64,30.55,XY24,BTT,STEEL
88,UNIONBANK,123.87,163.00,-1.39,M-LC,25.00,173074.0,32234.0,12254.0,-0.90,22.89,7.08,31.59,89.0,2.63,1.27,50.34,XY24,NTT,BANKS
70,SATIN,199.77,274.00,-15.90,H-SC,83.33,165102.0,-30872.0,103684.0,-0.81,-15.75,62.80,37.16,130.0,-0.30,1.21,23.30,XY24,NTT,FINANCE


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,51.69,M-SC,87.50,131544.0,-10935.0,11550.0,-0.63,-7.67,8.78,0.44,243.0,-0.95,0.96,55.26,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-214.35,H-SC,96.88,82534.0,-12782.0,12818.0,0.50,-13.41,15.53,0.04,154.0,-1.00,0.60,62.79,OX40N,NTT,MISC
81,TATAELXSI,7332.28,7332.0,-15.80,X-MC,28.12,88319.0,-14333.0,14325.0,-0.02,-13.96,16.22,-0.00,58.0,-1.00,0.65,34.26,OX40N,NTT,IT
53,KANSAINER,299.63,340.0,-67.47,H-SC,35.42,226350.0,-43317.0,79653.0,-0.26,-16.06,35.19,13.47,142.0,-0.54,1.66,13.89,XY24,NTT,PAINTS
46,INDIGOPNTS,1407.73,1408.0,121.54,H-SC,81.25,145303.0,-29256.0,29293.0,0.83,-16.76,20.16,0.02,127.0,-1.00,1.06,26.00,OX40N,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,1.04,43274.0,2019.0,8785.0,0.75,4.89,20.30,26.19,1.0,0.23,0.32,0.75,XY24,ATH,IT
83,TCS,3794.03,4998.00,-17.10,X-LC,40.62,270535.0,-25399.0,119306.0,0.18,-8.58,44.10,31.73,1.0,-0.21,1.98,8.28,X40,BTT,IT
48,INFY,1461.46,2275.00,-10.62,X-LC,54.17,287134.0,25533.0,120079.0,0.14,9.76,41.82,55.67,2.0,0.21,2.10,16.46,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-17.84,X-LC,18.75,257365.0,-12982.0,47613.0,0.14,-4.80,18.50,12.81,6.0,-0.27,1.88,7.64,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-9.04,X-LC,48.96,288405.0,22979.0,33916.0,-0.01,8.66,11.76,21.44,9.0,0.68,2.11,15.39,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,-1.39,M-LC,25.00,173074.0,32234.0,12254.0,-0.90,22.89,7.08,31.59,89.0,2.63,1.27,50.34,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.76,M-LC,26.04,103078.0,24716.0,9318.0,0.11,31.54,9.04,43.43,77.0,2.65,0.75,38.54,XY24,NTT,MISC
32,ENRIN,1377.95,2980.0,NaN,NaN,78.12,74500.0,40051.0,0.0,0.14,116.26,0.00,116.26,NaN,inf,0.54,13.92,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,KPIGREEN,497.21,732.16,29.90,H-SC,73.96,132917.0,7620.0,51585.0,2.49,6.08,38.81,47.25,139.0,0.15,0.97,66.29,MH,ATH,POWER
20,CAMS,3643.00,5250.99,6.54,H-SC,55.21,119753.0,17749.0,27280.0,-0.31,17.40,22.78,44.14,124.0,0.65,0.88,36.47,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.76,M-LC,26.04,103078.0,24716.0,9318.0,0.11,31.54,9.04,43.43,77.0,2.65,0.75,38.54,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,-1.39,M-LC,25.00,173074.0,32234.0,12254.0,-0.90,22.89,7.08,31.59,89.0,2.63,1.27,50.34,XY24,NTT,BANKS
95,WIPRO,243.46,420.00,-3.31,M-LC,76.04,165633.0,14688.0,94775.0,0.44,9.73,57.22,72.51,69.0,0.15,1.21,14.09,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2980.00,NaN,NaN,78.12,74500.0,40051.0,0.0,0.14,116.26,0.00,116.26,NaN,inf,0.54,13.92,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-37.67,M-SC,91.67,205418.0,33797.0,128263.0,-0.29,19.69,62.44,94.43,198.0,0.26,1.50,51.61,XR,NTT,DURABLES
54,KPIGREEN,497.21,732.16,29.90,H-SC,73.96,132917.0,7620.0,51585.0,2.49,6.08,38.81,47.25,139.0,0.15,0.97,66.29,MH,ATH,POWER
95,WIPRO,243.46,420.00,-3.31,M-LC,76.04,165633.0,14688.0,94775.0,0.44,9.73,57.22,72.51,69.0,0.15,1.21,14.09,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-46.58,H-SC,92.71,139284.0,15948.0,117820.0,1.15,12.93,84.59,108.45,122.0,0.14,1.02,38.17,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2980.00,NaN,NaN,78.12,74500.0,40051.0,0.0,0.14,116.26,0.00,116.26,NaN,inf,0.54,13.92,AR,ATH,ELECTRICAL
24,COFFEEDAY,59.14,80.00,-49.70,L-SC,100.00,75744.0,-37805.0,77857.0,5.34,-33.29,102.79,35.27,261.0,-0.49,0.55,85.39,XR,NTT,HOTELS
58,MASFIN,326.60,402.39,-18.70,H-SC,62.50,92325.0,-5655.0,28390.0,-0.10,-5.77,30.75,23.21,137.0,-0.20,0.68,33.90,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,9.28,X-SC,98.96,101682.0,719.0,93812.0,-0.14,0.71,92.26,93.63,64.0,0.01,0.74,51.45,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.76,M-LC,26.04,103078.0,24716.0,9318.0,0.11,31.54,9.04,43.43,77.0,2.65,0.75,38.54,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-49.70,L-SC,100.00,75744.0,-37805.0,77857.0,5.34,-33.29,102.79,35.27,261.0,-0.49,0.55,85.39,XR,NTT,HOTELS
4,ALKYLAMINE,2347.98,4546.37,9.28,X-SC,98.96,101682.0,719.0,93812.0,-0.14,0.71,92.26,93.63,64.0,0.01,0.74,51.45,SR,ATH,CHEMICALS
52,JPPOWER,18.73,26.20,-30.52,L-SC,97.92,139949.0,-1893.0,58457.0,0.98,-1.33,41.77,39.88,259.0,-0.03,1.02,41.83,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-214.35,H-SC,96.88,82534.0,-12782.0,12818.0,0.50,-13.41,15.53,0.04,154.0,-1.00,0.60,62.79,OX40N,NTT,MISC
72,SFL,1016.35,1287.00,42.51,H-SC,95.83,188069.0,-74149.0,143986.0,0.32,-28.28,76.56,26.63,144.0,-0.51,1.38,16.93,XY24,NTT,MISC


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.96
1,25,41.95
2,50,70.36


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.53
LC    34.16
MC    21.34
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.47
X40      12.23
XY25     12.07
X40N     11.76
XR       10.20
AR        8.34
OX40N     8.13
MH        1.71
X5K       1.47
X200      1.40
SR        1.23
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.70
M-SC    15.99
H-LC    14.22
X-LC    12.22
H-MC     9.43
X-MC     7.54
M-LC     6.66
X-SC     4.35
M-MC     4.00
L-SC     2.49
L-LC     1.06
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.15,-1.05,34.83
IT,12.08,-10.03,64.08
FINANCE,7.98,-10.73,50.81
BANKS,7.93,-5.20,48.13
MISC,6.65,-14.37,67.33
PAINTS,5.66,-12.98,35.25
ELECTRICAL,5.54,2.72,42.72
HEALTHCARE,4.51,-4.56,35.10
AUTO,4.35,-13.84,63.90


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2702463.0,27
XR,1065452.0,13
AR,1003780.0,10
X40,647777.0,9
X40N,557506.0,11
XY25,469913.0,8
OX40N,316103.0,11
SR,174838.0,2
MH,81724.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1956484.0,20
M-SC,1755964.0,21
H-LC,598018.0,12
H-MC,545867.0,8
X-LC,504692.0,7
X-MC,422508.0,6
M-MC,422095.0,3
X-SC,323235.0,5
M-LC,263036.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      851287.0      6
M-SC       XY24      810641.0      7
H-SC       AR        514934.0      3
M-SC       XR        330346.0      4
M-MC       XY24      312078.0      2
H-MC       XY24      298324.0      3
X-LC       X40       239385.0      2
H-SC       X40N      232187.0      4
           XR        216069.0      3
H-LC       X40N      194432.0      4
M-SC       AR        179765.0      2
           OX40N     172199.0      5
M-LC       XY24      156936.0      3
H-LC       X40       156745.0      3
X-MC       X40       150048.0      2
L-SC       XR        138779.0      2
X-SC       XR        122866.0      1
X-LC       AR        116800.0      1
M-MC       XR        110017.0      1
M-SC       XY25      109696.0      1
X-LC       XY25      109267.0      3
X-MC       XY25      105350.0      1
           XY24       96284.0      1
M-LC       XR         94775.0      1
X-SC       SR         93812.0      1
H-SC       OX40N      90422.0      3
H-MC       XY25       90234.0      1
M-SC       SR         81026.0      1
           X40        72291.0      1
H-LC       AR         70540.0      1
           X200       68743.0      1
X-SC       XY24       67400.0      1
H-LC       X5K        67173.0      2
H-MC       AR         62716.0      1
L-SC       AR         59025.0      1
           XY24       58457.0      1
X-MC       X40N       56501.0      1
L-MC       XR         52600.0      1
H-SC       MH         51585.0      1
L-LC       XY25       44041.0      1
H-LC       XY24       40385.0      1
X-LC       X40N       39240.0      1
X-SC       OX40N      39157.0      2
H-MC       X40N       35146.0      1
           MH         30139.0      1
           X40        29308.0      1
X-MC       OX40N      14325.0      1
M-LC       XY25       11325.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
